In [76]:
import os
import yt
import numpy as np

yt.enable_parallelism()

yt : [ERROR    ] 2024-02-24 03:26:48,561 Could not enable parallelism: only one mpi process is running. To remedy this, launch the Python interpreter as
  mpirun -n <X> python3 <yourscript>.py  # with X > 1 


False

In [69]:
downs_file_path = '/home/ivan/Study/Astro/solar/rad_transfer/datacubes/subs_3_flarecs-id_0012.h5'
subs_ds = yt.load(downs_file_path)
'''
# Print field list
subs_ds.field_list
subs_ds.derived_field_list
'''

yt : [INFO     ] 2024-02-24 03:20:53,069 Parameters: current_time              = 1.200143 code_time
yt : [INFO     ] 2024-02-24 03:20:53,072 Parameters: domain_dimensions         = [193 192  96]
yt : [INFO     ] 2024-02-24 03:20:53,075 Parameters: domain_left_edge          = [-0.5   0.   -0.25] code_length
yt : [INFO     ] 2024-02-24 03:20:53,077 Parameters: domain_right_edge         = [0.50000016 0.99999994 0.24999997] code_length
yt : [INFO     ] 2024-02-24 03:20:53,080 Parameters: cosmological_simulation   = 0


'\n# Print field list\nsubs_ds.field_list\nsubs_ds.derived_field_list\n'

In [64]:
# im, sc = yt.volume_render(subs_ds, ('gas', 'density'), fname='/home/ivan/Study/Astro/solar/rad_transfer/img/volumetric_demo/rendering.png')

In [75]:
sc = yt.create_scene(subs_ds, ('gas', 'density'), lens_type='perspective')
#sc.camera.width = (400, 'Mm')
source = sc[0]
source.tfh.set_bounds((1e8, 1e10))
source.tfh.set_log(True)
# source.tfh.grey_opacity = False

source.transfer_function = yt.ColorTransferFunction(
    np.log10((1e8, 1e10)), grey_opacity=True
)

def linramp(vals, minval, maxval):
    return (vals - vals.min()) / (vals.max() - vals.min())

source.transfer_function.map_to_colormap(
    np.log10(1e8), np.log10(1e10), colormap="cmyt.arbre", scale_func=linramp
)

#source.set_use_ghost_zones(True)
sc.camera.focus = subs_ds.domain_center

#sc.camera.normal_vector = [0., 0., 1.]
north_vector = [0., 1., 0.]
sc.camera.switch_orientation() # (normal_vector=normal_vector, north_vector=north_vector)
sc.save()
source.tfh.plot('transfer_function.png', profile_field=('gas', 'density'))

yt : [WARNING  ] 2024-02-24 03:25:24,314 No previously rendered image found, rendering now.
yt : [INFO     ] 2024-02-24 03:25:24,316 Rendering scene (Can take a while).
yt : [INFO     ] 2024-02-24 03:25:24,320 Creating volume
yt : [INFO     ] 2024-02-24 03:25:24,727 Saving rendered image to subs_3_flarecs-id_0012.h5_Render_density.png


In [100]:
repr((subs_ds.domain_center))

"unyt_array([ 8.20000000e-08,  4.99999968e-01, -1.60000000e-08], 'code_length')"

In [109]:
import unyt
sc = yt.create_scene(subs_ds, lens_type='perspective')
sc.camera.width = (2.0, 'code_length')
source = sc[0]

source.set_field(('gas', 'density'))
source.set_log(True)
bounds = (1e8, 1e10)

tf = yt.ColorTransferFunction(np.log10(bounds))
tf.add_gaussian(np.log10(1e9), width=0.005, height=[0.753, 1.0, 0.933, 1.0])

source.tfh.tf = tf
source.tfh.bounds = bounds

sc.camera.focus = unyt.unyt_array([8.20000000e-08, 0.25, -1.60000000e-08], 'Mm') # , 'code_length')  # subs_ds.domain_center
sc.camera.north_vector = [0, -1., 0.]
sc.camera.normal_vector = [-1., -1., -1.]
sc.camera.switch_orientation()

source.tfh.plot("transfer_function_new.png", profile_field=('gas', 'density'))
sc.save('rendering.png', sigma_clip=6)

yt : [WARNING  ] 2024-02-24 03:52:41,154 No previously rendered image found, rendering now.
yt : [INFO     ] 2024-02-24 03:52:41,155 Rendering scene (Can take a while).
yt : [INFO     ] 2024-02-24 03:52:41,160 Creating volume
yt : [INFO     ] 2024-02-24 03:53:05,928 Saving rendered image to rendering.png


In [67]:
sc.save()
?sc.camera.switch_orientation

yt : [WARNING  ] 2024-02-24 03:18:33,196 Previously rendered image exists, but rendering anyway. Supply 'render=False' to save previously rendered image directly.
yt : [INFO     ] 2024-02-24 03:18:33,199 Rendering scene (Can take a while).
yt : [INFO     ] 2024-02-24 03:18:56,279 Saving rendered image to subs_3_flarecs-id_0012.h5_Render_density.png
